# Turn aggregated data into weekly-behavior data

In [1]:
import pandas as pd
import numpy as np

In [2]:
zoom = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\student-dropout-analysis\data_processed\final_zoom.xlsx", dtype={'SID':str})
zoom

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,Grade,SID_full,Name,EFL,Note,MVA/MVSM,concat,4 last digits,Học sinh cùng lớp có chung 4 số cuối ID,_merge
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,Grade 1,2300-1816,Mai Nguyễn Trúc Linh,EFL K5,Chính thức,MVA,MVA2300-1816,1816.0,NaN,both
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,Grade 1,2200-2469,Nguyễn Phạm Nhật Minh,EFL K5,Chính thức,MVA,MVA2200-2469,2469.0,NaN,both
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,Grade 1,2400-2190,VY HÀ TÚ YÊN,EFL K5,Chính thức,MVA,MVA2400-2190,2190.0,NaN,both
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Lai Ngan Khanh 01771,23001771@minhvietacademy.org,...,Grade 1,2300-1771,Lại Ngân Khánh,EFL K5,Chính thức,MVA,MVA2300-1771,1771.0,NaN,both
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Tran Minh Khang 03927,23003927@minhvietacademy.org,...,Grade 1,2300-3927,Trần Minh Khang,EFL K5,Chính thức,MVA,MVA2300-3927,3927.0,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271177,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Hung, Viet",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271178,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Than Thu, Nguyen Minh",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271179,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Tien, Anh",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271180,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,phạm nguyên phúc hưng 02068,NaN,...,Grade 9-12,2400-2068,Phạm Nguyên Phúc Hưng,EFL 69,Chính thức,MVA,MVA2400-2068,2068.0,NaN,both


#### Aggregrate disconnected sessions
As students sometimes have internet issues and got disconnected from zoom, their participation is splitted into smaller sessions/records that seem like disengagement. So here we group them back together

In [4]:
#zoom events (ze)
ze = zoom.copy()

ze['Start time'] = pd.to_datetime(ze['Start time'], errors='coerce')
ze['End time'] = pd.to_datetime(ze['End time'], errors='coerce')
ze['Join time'] = pd.to_datetime(ze['Join time'], errors='coerce')
ze['Leave time'] = pd.to_datetime(ze['Leave time'], errors='coerce')


ze = (ze.groupby(['SID', 'Topic', 'Start time'])
              .agg(
                  end_time=('End time', 'max'),
                  student_duration=('student_stay_duration', 'sum'),
                  join_time=('Join time', 'min'),
                  leave_time=('Leave time', 'max'),
                  grade=('Grade', 'last'),
                  duration=('Duration (minutes)', 'first')
              )).reset_index()

ze['date'] = ze['Start time'].dt.floor("D")

ze

,SID,Topic,Start time,end_time,student_duration,join_time,leave_time,grade,duration,date
0,00000006,2506ELA,2025-02-24 18:24:00,2025-02-24 19:26:00,61,2025-02-24 18:24:00,2025-02-24 19:24:00,Grade 6,63,2025-02-24
1,00000006,2506ELA,2025-02-26 18:25:00,2025-02-26 19:26:00,45,2025-02-26 18:42:00,2025-02-26 19:26:00,Grade 6,62,2025-02-26
2,00000006,2506ELA,2025-03-03 18:26:00,2025-03-03 19:26:00,60,2025-03-03 18:27:00,2025-03-03 19:26:00,Grade 6,61,2025-03-03
3,00000006,2506ELA,2025-03-05 18:25:00,2025-03-05 19:27:00,9,2025-03-05 19:16:00,2025-03-05 19:25:00,Grade 6,63,2025-03-05
4,00000006,2506ELA,2025-03-17 18:27:00,2025-03-17 19:26:00,59,2025-03-17 18:27:00,2025-03-17 19:26:00,Grade 6,60,2025-03-17
...,...,...,...,...,...,...,...,...,...,...
207563,25000506,2501HUM,2025-05-27 19:25:00,2025-05-27 20:25:00,30,2025-05-27 19:55:00,2025-05-27 20:25:00,Grade 1,60,2025-05-27
207564,25000506,2501HUM,2025-05-29 19:27:00,2025-05-29 20:25:00,55,2025-05-29 19:30:00,2025-05-29 20:25:00,Grade 1,59,2025-05-29
207565,25000506,2501MAT,2025-05-14 19:26:00,2025-05-14 20:24:00,58,2025-05-14 19:26:00,2025-05-14 20:24:00,Grade 1,58,2025-05-14
207566,25000506,2501MAT,2025-05-21 19:26:00,2025-05-21 20:28:00,60,2025-05-21 19:28:00,2025-05-21 20:28:00,Grade 1,62,2025-05-21


#### Compute Minutes late / Minutes leaving early / Absent flag

In [19]:
grace_min = 5 #the school allows for 5 minutes grace

ze['minutes_late'] = ((ze['join_time'] - ze['Start time']).dt.total_seconds()/60) - grace_min
ze['minutes_late'] = ze['minutes_late'].clip(lower=0)

ze['minutes_left_early'] = ((ze['end_time'] - ze['leave_time']).dt.total_seconds()/60)
ze['minutes_left_early'] = ze['minutes_left_early'].clip(lower=0)

ze['attended_ratio'] = (ze['student_duration']/ze['duration'])
ze['very_short_flag'] = (ze['student_duration']<15).astype(int)
ze['partial_present'] = (ze['attended_ratio'] < 0.75).astype(int)

ze.head()

,SID,Topic,Start time,end_time,student_duration,join_time,leave_time,grade,duration,date,minutes_late,minutes_left_early,attended_ratio,is_present,very_short_flag,partial_present
0,00000006,2506ELA,2025-02-24 18:24:00,2025-02-24 19:26:00,61,2025-02-24 18:24:00,2025-02-24 19:24:00,Grade 6,63,2025-02-24,0.0,2.0,0.968254,1,0,0
1,00000006,2506ELA,2025-02-26 18:25:00,2025-02-26 19:26:00,45,2025-02-26 18:42:00,2025-02-26 19:26:00,Grade 6,62,2025-02-26,12.0,0.0,0.725806,1,0,1
2,00000006,2506ELA,2025-03-03 18:26:00,2025-03-03 19:26:00,60,2025-03-03 18:27:00,2025-03-03 19:26:00,Grade 6,61,2025-03-03,0.0,0.0,0.983607,1,0,0
3,00000006,2506ELA,2025-03-05 18:25:00,2025-03-05 19:27:00,9,2025-03-05 19:16:00,2025-03-05 19:25:00,Grade 6,63,2025-03-05,46.0,2.0,0.142857,0,1,1
4,00000006,2506ELA,2025-03-17 18:27:00,2025-03-17 19:26:00,59,2025-03-17 18:27:00,2025-03-17 19:26:00,Grade 6,60,2025-03-17,0.0,0.0,0.983333,1,0,0


#### Aggregrate to day-level
As students can have multiple sessions per day, here we aggregrate to form the basis for week-level

In [ ]:
day = (ze.groupby(['SID', 'date'])
       .agg(
           late_max=('minutes_late', 'max'), #max minutes late in that day
           late_mean=('minutes_late', 'mean'), #mean minutes late (over all the sessions in that day)
           late_sum=('minutes_late', 'sum'), #total minutes late in that day
           early_max=('minutes_left_early', 'max'), #max minutes left_early in that day
           early_mean=('minutes_left_early', 'mean'),#mean minutes left early (over all the sessions in that day)
           early_sum=('minutes_left_early', 'sum'), #total minutes left early in that day
           very_short_any=('very_short_flag', 'max'), #if there is 1 session attended under 15 minutes in that day or not 
           very_short_mean=('very_short_flag', 'mean'), #mean sessions atteneded under 15 minutes in that day
           very_short_sum=('very_short_flag', 'sum'), #total sessions atteneded under 15 minutes in that day
           partial_any=('partial_present', 'max'), #if there is 1 partially attended in that day or not 
           partial_mean=('partial_present', 'mean'), #mean sessions partially atteneded in that day
           partial_sum=('partial_present', 'sum'), #total sessions partially atteneded in that day
           attended_ratio_mean=('attended_ratio', 'mean'), #mean attended rate in that day
           attended_ratio_min=('attended_ratio', 'min'), #lowest attended rate in that day
           minutes_attended_day=('student_duration', 'sum'), #total minutes joined in that day
           total_duration_day=('duration', 'sum') #total scheduled minutes of classes in that day
       )).reset_index()

day['attended_ratio'] = day['minutes_attended_day']/day['total_duration_day'].replace(0,np.nan)
day.head()

,SID,date,late_max,late_mean,late_sum,early_max,early_mean,early_sum,very_short_any,very_short_mean,very_short_sum,partial_any,partial_mean,partial_sum,attended_ratio_mean,attended_ratio_min,minutes_attended_day,total_duration_day,attended_ratio
0,00000006,2025-02-20,0.0,0.0,0.0,14.0,7.0,14.0,0,0.0,0,1,0.5,1,0.872881,0.745763,106,121,0.876033
1,00000006,2025-02-24,0.0,0.0,0.0,11.0,6.5,13.0,0,0.0,0,0,0.0,0,0.885766,0.803279,110,124,0.887097
2,00000006,2025-02-25,3.0,1.5,3.0,7.0,4.5,9.0,0,0.0,0,0,0.0,0,0.860656,0.836066,105,122,0.860656
3,00000006,2025-02-26,12.0,6.0,12.0,17.0,8.5,17.0,0,0.0,0,1,1.0,2,0.707166,0.688525,87,123,0.707317
4,00000006,2025-02-27,0.0,0.0,0.0,20.0,20.0,20.0,0,0.0,0,1,1.0,1,0.590164,0.590164,36,61,0.590164


#### Convert day -> week statistics

In [22]:
day.columns

Index(['SID', 'date', 'late_max', 'late_mean', 'late_sum', 'early_max',
       'early_mean', 'early_sum', 'very_short_any', 'very_short_mean',
       'very_short_sum', 'partial_any', 'partial_mean', 'partial_sum',
       'attended_ratio_mean', 'attended_ratio_min', 'minutes_attended_day',
       'total_duration_day', 'attended_ratio'],
      dtype='object')

In [ ]:
to_week = lambda d:d.to_period("W-MON").start_time
day['week_start'] = day['date'].map(to_week)

weekly_stats = (day.groupby('SID', 'week_start')
                .agg(
                    avg_minutes_late=('minutes_late', 'mean'),
                    avg_minutes_left_early=('minutes_left_early','mean'),
                    total_minutes_attended=('minutes_attended_day', 'sum'),
                    total_planned_minutes=('total_duration_day', 'sum'),
                    present_sessions=('')
                )
                )